In [1]:
import os

HOME_PATH = os.getcwd()
root = 'FOV0'
print('HOME_PATH = ', HOME_PATH)
print('root: ', root)

HOME_PATH =  /Users/keith/astr/research_astr/summer-roman-project/FOV0_extraction/copying_pipeline
root:  FOV0


In [2]:
import importlib
import sys
import grizli

print(sys.version + "\n")

for module in ['grizli','grizli_aws', 'eazy', 'reprocess_wfc3', 'tristars', 'mastquery', 
               'wfc3dash', 'prospect', 'sep', 'numpy', 'scipy', 'astropy', 'astroquery', 
               'shapely', 'photutils', 'drizzlepac', 'wfc3tools', 'stsci.tools']:

    try:
        mod = importlib.import_module(module)
        print('{0:>20} : {1}'.format(module, mod.__version__))
    except ModuleNotFoundError:
        print('{0:>20} : {1}'.format(module, '*failed*'))

3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:35:20) [Clang 16.0.6 ]

              grizli : 1.11.10
          grizli_aws : *failed*
                eazy : 0.6.8
      reprocess_wfc3 : 0.2.3
            tristars : 0.1
           mastquery : 1.8.2
            wfc3dash : 0.1
            prospect : 1.3.0
                 sep : 1.2.1
               numpy : 1.26.4
               scipy : 1.13.0
             astropy : 6.0.1
          astroquery : 0.4.7
             shapely : 2.0.4
           photutils : 1.12.0


The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback  

In [3]:
%matplotlib inline

In [4]:
import glob
import time
import os
import sys
import yaml

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

from astropy.io import fits
import drizzlepac

import grizli
from grizli.pipeline import auto_script
from grizli import utils, fitting, multifit, prep
utils.set_warnings()



In [5]:
default_kwargs = auto_script.get_yml_parameters(local_file='my_params.yml', 
                                                copy_defaults=True)
                        
kwargs = auto_script.get_yml_parameters(local_file='my_params.yml', 
                                        copy_defaults=False)

Copied default parameter file to my_params.yml


In [6]:
print(os.getcwd(), '\n', root)

/Users/keith/astr/research_astr/summer-roman-project/FOV0_extraction/copying_pipeline 
 FOV0


In [7]:
os.chdir(os.path.join(HOME_PATH, root, 'Prep'))
kwargs['parse_visits_args']['combine_same_pa'] = True

visits, all_groups, info = auto_script.parse_visits(field_root=root,
                                                    filters="D1_",
                                                    **kwargs["parse_visits_args"])

# (2024-07-01 14:13:24.875)

auto_script.parse_visits(**{'files': [], 'field_root': 'FOV0', 'RAW_PATH': '../RAW', 'use_visit': True, 'combine_same_pa': True, 'combine_minexp': 2, 'is_dash': False, 'filters': 'D1_', 'max_dt': 0.5, 'visit_split_shift': 1.5, 'file_query': '*'})

 date-obs 
----------
2021-09-15
2021-09-15
2021-09-15
2021-09-15
indef-RS_-FO-000.0-D1_ 2
indef-ode-l_-000.0-D1_ 2
# (2024-07-01 14:13:24.902)
parse_visits(combine_same_pa=True), max_dt=0.5: BEFORE   2 visits
# (2024-07-01 14:13:24.904)
parse_visits(combine_same_pa=True), max_dt=0.5:  AFTER   2 visits
** Combine same PA: **
0 irs_-000.0-d1_ 2
1 iode-000.0-d1_ 2

 == Grism groups ==



In [8]:
# Parameter lists
visit_prep_args = kwargs['visit_prep_args']
preprocess_args = kwargs['preprocess_args']

# Maximum shift for "tweakshifts" relative alignment
tweak_max_dist = 1
if 'tweak_max_dist' not in visit_prep_args:
    visit_prep_args['tweak_max_dist'] = tweak_max_dist

# Fit and subtract a SExtractor-like background to each visit
visit_prep_args['imaging_bkg_params']  = {'bh': 256, 'bw': 256, 'fh': 3, 'fw': 3, 
                                          'pixel_scale': 0.1, 'get_median': False}

# Alignment reference catalogs, searched in this order
visit_prep_args['reference_catalogs'] = ['LS_DR9', 'PS1','GAIA','SDSS','WISE']

# LS_DR9 is DR9 of the NOAO legacysurveys
# https://datalab.noirlab.edu/ls/ls.php

In [9]:
#####################
### Alignment & mosaics    
os.chdir(os.path.join(HOME_PATH, root, 'Prep'))

# This script will do all the preprocessing of the grism *and* imaging visits 
# found in your archive query.
auto_script.preprocess(field_root=root, HOME_PATH=HOME_PATH,
                       visit_prep_args=visit_prep_args, **preprocess_args)    

# (2024-07-01 14:13:24.926)

auto_script.preprocess(**{'field_root': 'FOV0', 'HOME_PATH': '/Users/keith/astr/research_astr/summer-roman-project/FOV0_extraction/copying_pipeline', 'PERSIST_PATH': None, 'min_overlap': 0.2, 'make_combined': False, 'catalogs': ['PS1', 'DES', 'NSC', 'SDSS', 'GAIA', 'WISE'], 'use_visit': True, 'master_radec': None, 'parent_radec': None, 'use_first_radec': False, 'skip_imaging': False, 'clean': True, 'skip_single_optical_visits': False, 'visit_prep_args': {'align_thresh': None, 'align_rms_limit': 2, 'align_mag_limits': [14, 24, 0.05], 'align_assume_close': False, 'align_transform': None, 'align_ref_border': 100, 'align_guess': None, 'max_err_percentile': 99, 'align_min_flux_radius': 1.0, 'align_min_nexp': 2, 'align_final_niter': 8, 'catalog_mask_pad': 0.05, 'match_catalog_density': False, 'skymethod': 'localmin', 'drizzle_params': {}, 'single_image_CRs': True, 'run_tweak_align': True, 'tweak_threshold': 3.0, 'tweak_fit_order': -1, 'tweak_max_dist': 100, 'twea

In [10]:
!ls *sci.fits # individual drizzled visits

zsh:1: no matches found: *sci.fits
